## Dataset

In [1]:
%load_ext autoreload
%autoreload 2
import transform

vocab = "0123456789+="
char_lookup_table = transform.CharacterTable(vocab)
list(char_lookup_table.generate_samples(5))

RuntimeError: jaxlib version 0.3.20 is newer than and incompatible with jax version 0.3.15. Please update your jax and/or jaxlib packages.

In [3]:
batch = char_lookup_table.get_batch(12)
print(f"inputs shape: {batch['query'].shape}")
print(f"targets shape: {batch['answer'].shape}")
print(f"shape: batch_size, sequence_length, vocab_size")


inputs shape: (12, 8, 14)
targets shape: (12, 8, 14)
shape: batch_size, sequence_length, vocab_size


In [4]:
import jax.numpy as jnp

labels = batch['answer']
eos_row = labels[:, :, char_lookup_table.eos_id] # as we are dealing with a one hot encoded variable and we now the eos_id, we can select the column
# representing the eos_id and find it's argmax. Because if we find it's argmax, we know everything after that index will be padded.

print(labels[0, :]) # looking at a single sample it becomes more obvious. The eos_id = 1, in addition, the pad token is 0. 
# so if you look down the first column until you see the first 1 (i.e. the row containing the eos token), than we know that everything after 
# that row AND in the pad token column (i.e. column 0) should be equal to 1.


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [5]:
eos_idx = jnp.argmax(eos_row, axis=-1) # get the first occurence when we get the end of sentence id


In [6]:
eos_idx

DeviceArray([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [7]:

jnp.where(
    eos_row[jnp.arange(eos_row.shape[0]), eos_idx],
    eos_idx + 1, # the +1 makes sure we include the eos token, which will also be needed during inference to know when to stop
    labels.shape[1] # if no eos id is present, use the entire sequence as target
)

DeviceArray([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int32)

lets take an example. If the inputs = `120+123` than the target = `=243`
in addition, lets say we have the following char_to_idx mapping: 
```
{
    0: '_',
    1: '|',
    2: '+',
    3: '0',
    4: '1',
    5: '2',
    6: '3',
    7: '4',
    8: '5',
    9: '6',
    10: '7',
    11: '8',
    12: '9',
    13: '='
 }
```
the target becomes 
```
[
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], <-- =
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], <-- 2
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], <-- 4
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], <-- 3
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- |
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], <-- _
]
```

In [8]:
labels[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [9]:
import jax.numpy as jnp

from transform import get_sequence_lengths

def mask_sequences(sequence_batch: jnp.ndarray, lengths: jnp.ndarray) -> jnp.ndarray:
  """Sets positions beyond the length of each sequence to 0."""
  return sequence_batch * (
      lengths[:, jnp.newaxis] > jnp.arange(sequence_batch.shape[1])[jnp.newaxis])[..., jnp.newaxis]

lengths = get_sequence_lengths(labels, char_lookup_table.eos_id)
masked_labels = mask_sequences(labels, lengths)


# note that in the masked_labels, all padded rows are set to 0
# whereas the labels still contain the padded one_hot encoded elements
masked_labels[0], labels[0]



(DeviceArray([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],            dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [10]:

labels.shape
lengths = get_sequence_lengths(labels, char_lookup_table.eos_id)
lengths_expanded = lengths[:, jnp.newaxis] # from shape (batch_size,) to (batch_size, 1)
jnp.arange(labels.shape[1]) # array for each element in the sequence. shape = (sequence_length,)
sequence_expanded = jnp.arange(labels.shape[1])[jnp.newaxis] # array for each element in the sequence. shape = (1, sequence_length)

# shapes are (batch_size, 1), (1, sequence_length)
print(lengths_expanded.shape, sequence_expanded.shape)

# if we compare them using the > operation, we are creating a boolean matrix
# which returns true for each element where the sequence length > labels index
print(lengths_expanded)
print(sequence_expanded)
print(1.0 * (lengths_expanded > sequence_expanded)) # multiply by 1.0 to turn boolean into float
masks = 1.0 * (lengths_expanded > sequence_expanded)
# masks.shape = (batch_size, sequence_size)
masks.shape 

(12, 1) (1, 8)
[[5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]]
[[0 1 2 3 4 5 6 7]]
[[1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]]


(12, 8)

# The Model


inputs --> encoder --> embedding_representation_input_domain --> decoder --> embedding_representation_target_domain --> dense --> predictions


In [11]:
import flax.linen as nn

my interpretation of the nn.scan functionality is as follows
- nn.scan can be seen as a for loop with state carried over at each step
- the first argument is the operation that we want to apply (in our case, we want to feed each step through the lstmcell, so the operation is the nn.LSTMCell).
- We specify the variable_broadcast. This value is shared across each step and has no dependencies. Recall that the weights of an LSTMCell are shared. As such, we can provide these weights at each time step. Stated differently, we are broadcasting them at each time step
- we set split_rngs for the params to false. We don't want to create random parameters at each time step
- now lets move over to the axes. In the aforementioned documentation, I'm constantly talking about timesteps. The axes is what allows me to talk about it in this way. Remember, the inputs are of shape (batch_size, timesteps, n_features). by specifying in_axis=1, we are saying, apply this function over the first dimension (i.e. the timesteps/sequence axis).
- out_axis specifies the axis where the result will be added to (i.e. the dimension to place the results). As such, the output will have an extra dimension/axis at the position indicated by the `out_axis` (e.g. in the case of one sample with 10 output features, the shape would be (1, 10). with out_axis=1 the result would become (1, 1, 10))
 

In [12]:
# Example taken from flax documentation
# --> https://flax.readthedocs.io/en/latest/api_reference/_autosummary/flax.linen.scan.html?highlight=flax%20scan
import flax
import flax.linen as nn
from jax import random

class SimpleScan(nn.Module):
  @nn.compact
  def __call__(self, c, xs):
    LSTM = nn.scan(nn.LSTMCell,
                   variable_broadcast="params",
                   split_rngs={"params": False},
                   in_axes=1,
                   out_axes=1)
    return LSTM()(c, xs)

batch_size, seq_len, in_feat, out_feat = 16, 20, 3, 5
key_1, key_2, key_3 = random.split(random.PRNGKey(0), 3)

xs = random.uniform(key_1, (batch_size, seq_len, in_feat))
init_carry = nn.LSTMCell.initialize_carry(key_2, (batch_size,), out_feat)

model = SimpleScan()
variables = model.init(key_3, init_carry, xs)
out_carry, out_val = model.apply(variables, init_carry, xs)

assert out_val.shape == (batch_size, seq_len, out_feat)
out_val.shape 
# for each sample in the the batch (16 in total), at each time step (20 in total)
# we have the knowledge 'encoded' in a 5 dimensional vector


/home/vscode/.cache/pypoetry/virtualenvs/mlteacher-qwQzX-qT-py3.9/lib/python3.9/site-packages/flax/linen/transforms.py:138: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  new_args, new_kwargs = jax.tree_map(get_arg_scope, (args, kwargs))
/home/vscode/.cache/pypoetry/virtualenvs/mlteacher-qwQzX-qT-py3.9/lib/python3.9/site-packages/flax/linen/transforms.py:157: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  jax.tree_map(get_scopes_inner, attrs)
/home/vscode/.cache/pypoetry/virtualenvs/mlteacher-qwQzX-qT-py3.9/lib/python3.9/site-packages/flax/core/lift.py:112: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  scopes, treedef = jax.tree_flatten(scope_tree)
/home/vscode/.cache/pypoetry/virtualenvs/mlteacher-qwQzX-qT-py3.9/lib/python3.9/site-packages/flax/core/scope

(16, 20, 5)

In [13]:
init_carry[0].shape
out_carry[1].shape


(16, 5)

## Build the encoder part of the model

def select_carried_state(new_state, old_state):
    return jnp.where(is_eos[:, np.newaxis], old_state, new_state)



initial_

In [14]:
import functools
from typing import Any, Tuple

from flax import linen as nn
import jax
import jax.numpy as jnp
import numpy as np

Array = Any
PRNGKey = Any

class EncoderLSTM(nn.Module):
  """EncoderLSTM Module wrapped in a lifted scan transform."""
  eos_id: int

  @functools.partial(
      nn.scan,
      variable_broadcast='params',
      in_axes=1,
      out_axes=1,
      split_rngs={'params': False})
  @nn.compact
  def __call__(self, carry: Tuple[Array, Array],
               x: Array) -> Tuple[Tuple[Array, Array], Array]:
    """Applies the module."""
    lstm_state, is_eos = carry
    new_lstm_state, y = nn.LSTMCell()(lstm_state, x)
    # Pass forward the previous state if EOS has already been reached.
    def select_carried_state(new_state, old_state):
      return jnp.where(is_eos[:, np.newaxis], old_state, new_state)
    # LSTM state is a tuple (cell_state, hidden_state).
    # we need to select what state to carry for both 
    # the cell_state as the hidden_state
    carried_lstm_state = tuple(
        select_carried_state(*s) for s in zip(new_lstm_state, lstm_state))
    # Update `is_eos`.
    is_eos = jnp.logical_or(is_eos, x[:, self.eos_id])
    return (carried_lstm_state, is_eos), y

  @staticmethod
  def initialize_carry(batch_size: int, hidden_size: int) -> Tuple[Array, Array]:
    # Use a dummy key since the default state init fn is just zeros.
    return nn.LSTMCell.initialize_carry(
        jax.random.PRNGKey(0), (batch_size,), hidden_size)


class Encoder(nn.Module):
  """LSTM encoder, returning state after finding the EOS token in the input."""
  hidden_size: int
  eos_id: int

  @nn.compact
  def __call__(self, inputs: Array):
    # inputs.shape = (batch_size, seq_length, vocab_size).
    batch_size = inputs.shape[0]
    lstm = EncoderLSTM(name='encoder_lstm', eos_id=self.eos_id)
    init_lstm_state = lstm.initialize_carry(batch_size, self.hidden_size)
    # We use the `is_eos` array to determine whether the encoder should carry
    # over the last lstm state, or apply the LSTM cell on the previous state.
    init_is_eos = jnp.zeros(batch_size, dtype=bool)
    init_carry = (init_lstm_state, init_is_eos)
    (final_state, _), _ = lstm(init_carry, inputs)
    return final_state

In [15]:
nn.LSTMCell().initialize_carry(jax.random.PRNGKey(0), (16, ), 12)

(DeviceArray([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0

In [16]:
xs[:, eos_idx].shape

(16, 12, 3)

In [17]:
batch.keys()

dict_keys(['query', 'answer'])

In [18]:
batch = char_lookup_table.get_batch(16) 
print(batch['query'].shape)
print(batch['query'][:, eos_idx].shape)
print(char_lookup_table.decode_onehot(batch['query'][:1]))
print(batch['query'][0][:,eos_idx])
char_lookup_table._char_indices

# char_lookup_table.decode(batch['answer'])

(16, 8, 14)
(16, 12, 14)
['49+688']
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


{'+': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '=': 13}

In [19]:
# in the network we initialize the is_eos with all zeros
# then we compute the logical_or based on the input sequence where the id == eos_id
# Finally, we take the xor, updating the is_eos matrix to contain ones, where the 
# is_eos matrix is one or the input_matrix is one 
jnp.logical_or(jnp.array([1,0,1,0]), jnp.array([1,1,0, 0]))

DeviceArray([ True,  True,  True, False], dtype=bool)

In [20]:
batch_size = 32
hidden_size = 512
batch = char_lookup_table.get_batch(batch_size)

rng = jax.random.PRNGKey(0)

encoder_model = Encoder(hidden_size, char_lookup_table.eos_id)
example = jnp.ones((1, char_lookup_table.max_input_len, char_lookup_table.vocab_size), jnp.float32)
variables = encoder_model.init(rng, example)
model_params = variables['params']

In [21]:
batch['query'].shape

(32, 8, 14)

In [22]:

encoding = encoder_model.apply({
    "params": model_params
}, batch['query'])

In [23]:
encoding[0].shape, encoding[1].shape

((32, 512), (32, 512))

In [24]:
batch.keys()

dict_keys(['query', 'answer'])

# Decoder Model

In [25]:

class DecoderLSTM(nn.Module):
  """DecoderLSTM Module wrapped in a lifted scan transform.
  Attributes:
    teacher_force: See docstring on Seq2seq module.
    vocab_size: Size of the vocabulary.
  """
  teacher_force: bool
  vocab_size: int

  @functools.partial(
      nn.scan,
      variable_broadcast='params',
      in_axes=1,
      out_axes=1,
      split_rngs={'params': False, 'lstm': True})
  @nn.compact
  def __call__(self, carry: Tuple[Array, Array], x: Array) -> Array:
    """Applies the DecoderLSTM model."""
    lstm_state, last_prediction = carry
    if not self.teacher_force:
      x = last_prediction
    lstm_state, y = nn.LSTMCell()(lstm_state, x)
    logits = nn.Dense(features=self.vocab_size)(y)
    # Sample the predicted token using a categorical distribution over the
    # logits.
    categorical_rng = self.make_rng('lstm')
    predicted_token = jax.random.categorical(categorical_rng, logits)
    # Convert to one-hot encoding.
    prediction = jax.nn.one_hot(
        predicted_token, self.vocab_size, dtype=jnp.float32)

    return (lstm_state, prediction), (logits, prediction)


class Decoder(nn.Module):
  """LSTM decoder.
  Attributes:
    init_state: [batch_size, hidden_size]
      Initial state of the decoder (i.e., the final state of the encoder).
    teacher_force: See docstring on Seq2seq module.
    vocab_size: Size of the vocabulary.
  """
  init_state: Tuple[Any]
  teacher_force: bool
  vocab_size: int

  @nn.compact
  def __call__(self, inputs: Array) -> Tuple[Array, Array]:
    """Applies the decoder model.
    Args:
      inputs: [batch_size, max_output_len-1, vocab_size]
        Contains the inputs to the decoder at each time step (only used when not
        using teacher forcing). Since each token at position i is fed as input
        to the decoder at position i+1, the last token is not provided.
    Returns:
      Pair (logits, predictions), which are two arrays of respectively decoded
      logits and predictions (in one hot-encoding format).
    """
    lstm = DecoderLSTM(teacher_force=self.teacher_force,
                       vocab_size=self.vocab_size)
    init_carry = (self.init_state, inputs[:, 0])
    _, (logits, predictions) = lstm(init_carry, inputs)
    return logits, predictions

In [26]:

d = Decoder(init_state=encoding, teacher_force=False, vocab_size=char_lookup_table.vocab_size)

In [27]:
init_decoder_input = char_lookup_table.one_hot(char_lookup_table.encode('=')[0:1])

In [28]:

init_decoder_inputs = jnp.tile(init_decoder_input,
                                (batch['query'].shape[0], char_lookup_table.max_output_len, 1))

In [29]:
init_decoder_input.shape, init_decoder_inputs.shape

((1, 14), (32, 6, 14))

In [30]:
key_1, key_2 = jax.random.split(jax.random.PRNGKey(0))
decoder_params = d.init({"params":key_1, "lstm": key_2}, init_decoder_inputs)
# d.apply()

/home/vscode/.cache/pypoetry/virtualenvs/mlteacher-qwQzX-qT-py3.9/lib/python3.9/site-packages/flax/core/lift.py:42: FutureWarning: jax.tree_map is deprecated, and will be removed in a future release. Use jax.tree_util.tree_map instead.
  return jax.tree_map(


In [31]:
lstm_rng = jax.random.PRNGKey(0)
step = 200
# we need a random key as our decoder samples 
# using a categorical distibution based on the logits
lstm_key = jax.random.fold_in(lstm_rng, step) 

logits, predictions = d.apply(decoder_params, init_decoder_inputs, rngs={'lstm': lstm_key})

In [32]:
logits.shape, predictions.shape

((32, 6, 14), (32, 6, 14))

In [33]:
# model isn't trained, but here are the results (query, target, prediction)
print(char_lookup_table.decode_onehot(batch['query']))
print(char_lookup_table.decode_onehot(batch['answer']))
print(char_lookup_table.decode_onehot(predictions))

['98+971' '29+164' '52+142' '30+777' '94+319' '39+66' '19+235' '65+605'
 '45+234' '97+187' '58+886' '65+668' '65+848' '71+809' '24+852' '33+969'
 '41+536' '68+644' '87+204' '74+668' '34+367' '26+629' '81+135' '88+7'
 '92+227' '27+432' '86+383' '38+416' '63+351' '62+980' '95+734' '3+282']
['=1069' '=193' '=194' '=807' '=413' '=105' '=254' '=670' '=279' '=284'
 '=944' '=733' '=913' '=880' '=876' '=1002' '=577' '=712' '=291' '=742'
 '=401' '=655' '=216' '=95' '=319' '=459' '=469' '=454' '=414' '=1042'
 '=829' '=285']
['63' '0' '300=0=' '98' '10+316' '1734=4' '' '0' '=0_=+1' '620533'
 '435923' '7_94+' '30=4' '70=1+5' '6+9262' '583' '082297' '+30825'
 '_96=95' '2=6711' '408' '3+3' '25' '2_506+' '=56+31' '5_7510' '_31=83'
 '453736' '95+547' '5=_' '02' '_575=5']


# The model

In [34]:
class Seq2seq(nn.Module):

    teacher_force: bool
    hidden_size: int
    vocab_size: int
    eos_id: int = 1

    @nn.compact
    def __call__(self, encoder_inputs, decoder_inputs):
        initial_decoder_state = Encoder(
            hidden_size=self.hidden_size,
            eos_id=self.eos_id
        )(encoder_inputs)

        logits, predictions = Decoder(
            init_state=initial_decoder_state,
            teacher_force=self.teacher_force,
            vocab_size=self.vocab_size
        )(decoder_inputs[:, :-1])
        return logits, predictions


In [35]:
teacher_force = False
hidden_size = 512
eos_id = char_lookup_table.eos_id
vocab_size = char_lookup_table.vocab_size
model = Seq2seq(
    teacher_force=teacher_force,
    hidden_size=hidden_size,
    vocab_size=vocab_size
)

In [36]:
rng = jax.random.PRNGKey(0)
params_key, lstm_key = jax.random.split(rng)
max_input_sequence_len = char_lookup_table.max_input_len
max_output_sequence_len = char_lookup_table.max_output_len
variables = model.init(
    {
        "params": params_key,
        "lstm": lstm_key
    },
    jnp.ones((batch_size, max_input_sequence_len, vocab_size), dtype=jnp.float32),
    jnp.ones((batch_size, max_output_sequence_len, vocab_size), dtype=jnp.float32)
)

In [37]:
variables.keys()

frozen_dict_keys(['params'])

In [38]:
model.apply(variables, batch['query'], batch['answer'], rngs={
    "lstm": lstm_key
})

(DeviceArray([[[ 0.08307394, -0.06417654,  0.11232382, ...,  0.04796918,
                 0.08337983, -0.04808454],
               [-0.04887169,  0.02889351,  0.08376509, ..., -0.0573354 ,
                 0.02569693, -0.01780605],
               [ 0.07617846, -0.06507105,  0.15715244, ...,  0.07221642,
                 0.12518902, -0.0459436 ],
               ...,
               [ 0.08691359, -0.09904549,  0.05844625, ...,  0.03085802,
                -0.13066357, -0.01850299],
               [ 0.17952195,  0.04801203,  0.10827539, ..., -0.02468754,
                 0.0370799 , -0.06572822],
               [ 0.03389015,  0.00849534,  0.0068223 , ...,  0.00689863,
                 0.0393358 , -0.11326101]],
 
              [[ 0.09038381, -0.08036304,  0.10141592, ...,  0.02821084,
                 0.10218308, -0.02035185],
               [ 0.13380422,  0.05083488,  0.04915942, ..., -0.10524669,
                 0.10610627,  0.05437368],
               [ 0.1284994 ,  0.04025817,  0.1039

In [42]:

# model.apply(variables, batch['query'], batch['answer'], rngs={
#     "lstm": lstm_key
# })
# during inference you provide the start_of_sentence token
# which in our case is the '=' character 
batch['answer'].shape # <-- input_shape

(32, 8, 14)

In [40]:
init_decoder_inputs.shape

(32, 6, 14)

In [41]:
init_decoder_inputs[0], char_lookup_table._char_indices

(DeviceArray([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],            dtype=float32),
 {'+': 2,
  '0': 3,
  '1': 4,
  '2': 5,
  '3': 6,
  '4': 7,
  '5': 8,
  '6': 9,
  '7': 10,
  '8': 11,
  '9': 12,
  '=': 13})

In [ ]:
from model import Seq2seq



## Training

# Saving

In [42]:

import functools
from typing import Callable, Dict, List

import absl

import flax
from flax import linen as nn
from flax.metrics import tensorboard
import flax.optim

import jax
from jax import numpy as jnp
from jax.experimental import jax2tf
from mlteacher.mlops.transform import CharacterTable
import numpy as np

import tensorflow as tf


# The transformed feature names

# Type abbreviations: (B is the batch size)
_Array = np.ndarray
_InputBatch = Dict[str,
                   _Array]  # keys are _FEATURE_KEYS_XF and values f32[B, 1]
_LogitBatch = _Array  # of shape f32[B, 3]
_LabelBatch = _Array  # of shape int64[B, 1]
_Params = Dict[str, _Array]


class _SavedModelWrapper(tf.train.Checkpoint):
  """Wraps a function and its parameters for saving to a SavedModel.
  Implements the interface described at
  https://www.tensorflow.org/hub/reusable_saved_models.
  This class contains all the code needed to convert a Flax model to a
  TensorFlow saved model.
  """

  def __init__(self,
               tf_graph: Callable[[_InputBatch], _Array],
               param_vars: Dict[str, tf.Variable]):
    """Builds the tf.Module.
    Args:
      tf_graph: a tf.function taking one argument (the inputs), which can be be
        tuples/lists/dictionaries of np.ndarray or tensors. The function may
        have references to the tf.Variables in `param_vars`.
      param_vars: the parameters, as tuples/lists/dictionaries of tf.Variable,
        to be saved as the variables of the SavedModel.
    """
    super().__init__()
    # Implement the interface from
    # https://www.tensorflow.org/hub/reusable_saved_models
    self.variables = tf.nest.flatten(param_vars)
    self.trainable_variables = [v for v in self.variables if v.trainable]
    self._tf_graph = tf_graph

  @tf.function
  def __call__(self, inputs):
    return self._tf_graph(inputs)




from mlteacher.mlops import train, models

def save_model():
    rng = jax.random.PRNGKey(0)
    ctable = CharacterTable("0123456789=+")
    model = models.Seq2seq(teacher_force=False, hidden_size=512, vocab_size=ctable.vocab_size)
    params = train.get_initial_params(model, rng, ctable)

    predict_fn = lambda params, input: model.apply({"params": params}, *input, rngs={"lstm":jax.random.PRNGKey(1)})
    tf_fn = jax2tf.convert(predict_fn, with_gradient=False, enable_xla=True)

    return tf_fn, params
tf_fn, trained_params = save_model()                        

In [45]:

param_vars = tf.nest.map_structure(
    # Due to a bug in SavedModel it is not possible to use tf.GradientTape
    # on a function converted with jax2tf and loaded from SavedModel.
    # Thus, we mark the variables as non-trainable to ensure that users of
    # the SavedModel will not try to fine tune them.
    lambda param: tf.Variable(param, trainable=False),
    trained_params)
tf_graph = tf.function(
    lambda inputs, decoder_inputs: tf_fn(param_vars, (inputs, decoder_inputs)),
    autograph=False,
    experimental_compile=True)

signatures = {}
# This signature is needed for TensorFlow Serving use.
batch_size = 1
ctable = CharacterTable("0123456789+=")
ctable.get_batch(1, 1)['query'].shape
input_signatures = [
            tf.TensorSpec((batch_size,) + (ctable.max_input_len, ctable.vocab_size), tf.float32),
            tf.TensorSpec((batch_size,) + (ctable.max_output_len, ctable.vocab_size), tf.float32)
          ]
signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = tf_graph.get_concrete_function(input_signatures[0], input_signatures[1])

tf_model = _SavedModelWrapper(tf_graph, param_vars)

In [6]:
from datetime import datetime

'20221010072911'

In [7]:
def save_model(model, signatures, serving_dir):
    latest_model = datetime.now().strftime("%Y%m%d%H%M%S")
    serving_dir = os.path.join(serving_dir, latest_model)
    tf.saved_model.save(tf_model, serving_dir, signatures=signatures)

In [47]:
loaded_model = tf.saved_model.load("./models")

In [55]:
serving_default = loaded_model.signatures['serving_default']

In [56]:

ctable = CharacterTable("0123456789=+")
b = ctable.get_batch(1, 2)
q, a = b['query'], b['answer']

In [58]:
serving_default(inputs=q, decoder_inputs=a)

{'output_1': <tf.Tensor: shape=(1, 5, 14), dtype=float32, numpy=
 array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]],
       dtype=float32)>,
 'output_0': <tf.Tensor: shape=(1, 5, 14), dtype=float32, numpy=
 array([[[ 0.08430967, -0.08283493,  0.09226608, -0.06462146,
           0.0793648 , -0.1541741 , -0.09239573,  0.00601145,
          -0.08144277,  0.11923195,  0.03178895,  0.04326432,
           0.1258326 , -0.0333593 ],
         [ 0.01941263,  0.0411636 ,  0.16464294, -0.03763947,
           0.10835177, -0.17228547, -0.13574198, -0.09845351,
          -0.01242627,  0.1081738 ,  0.10392632, -0.07437727,
           0.04082923, -0.06269556],
         [-0.04022854,  0.02152416,  0.06989108,  0.04568947,
 